# TP2 : Construir uma classe Python que implemente o ECDSA usando uma das curvas elípticas primas definidas no FIPS186-4 

A alinea 3 do segundo trabalho prático pretende que se implemente um mecanismo de assinatura digital, recorrendo
para isso às curvas elíticas. São oferecidas curvas para criptografia de 192, 224, 256, 384 e 521 bits definida no 
**FIPS186-4** cuja forma é E : y^2 = x^3 - 3x. Destas a que escolhemos foi a *P-224*. Depois de recolhermos a 
definição das mesmas, procedemos à implementação do algoritmo com base em documentos encontrados na internet.

De modo a facilitar a compreensão do algoritmo implementado, é explicado o significado das variáveis utilizadas:

* **s** : *Secret Key* (inteiro) gerada aleatoriamente
* **V** : A nossa *Public Key* (ponto da curva) obtida da multiplicação da *secret key* com *G*
* **G** : Ponto pertecente à nossa curva elitica (constituido por gx e gy)
* **p** : Primo que define o corpo finito da curva elitica
* **n** : Ordem do primo
* **e** : Chave (inteiro) gerada  aleatoriamente e de forma segura
* **(s1,s2)** : Assinatura do documento (inteiro,inteiro)

In [50]:
#tabelamento da curva P-224
NIST = dict()
NIST['P-224'] = { 
    'p': 26959946667150639794667015087019630673557916260026308143510066298881,
    'n': 26959946667150639794667015087019625940457807714424391721682722368061,
    'seed': 'bd71344799d5c7fcdc45b59fa3b9ab8f6a948bc5',
    'c': '5b056c7e11dd68f40469ee7f3c7a7d74f7d121116506d031218291fb',
    'b': 'b4050a850c04b3abf54132565044b0b7d7bfd8ba270b39432355ffb4',
    'Gx': 'b70e0cbd6bb4bf7f321390b94a03c1d356c21122343280d6115c1d21',
    'Gy': 'bd376388b5f723fb4c22dfe6cd4375a05a07476444d5819985007e34'
} 

class ECDSA:
    
    def __init__(self):
        c = NIST['P-224']
        p  = c['p'] 
        n  = c['n']
        b  = ZZ(c['b'],16)
        Gx = ZZ(c['Gx'],16)
        Gy = ZZ(c['Gy'],16)
        E = EllipticCurve(GF(p),[-3,b]) #curva eliptica
        self.G = E((Gx,Gy)) #Ponto pertecente à nossa curva elitica (constituido por gx e gy)
        self.s = ZZ.random_element(1,p-1) #selecionar numero aleatorio entre 1 e p-1 -secret key
        self.V= self.s*self.G

    def sign_key(self):
        return self.s
    
    def verify_key(self):
        return self.V, self.G
    
    
    def sign(self,key,msg):
        Q = GF(self.G.order())
        
        #hash
        Z = IntegerRing()
        m = Z(msg.encode("hex"), base=16)
        d = Q(m)
        
        e = Q.random_element()
        (x,y,z) = int(e) * self.G #calcular o ponto na curva E:(x,y) = k*G
        s1 = Q(x) #r= x (mod n)
        s2 = Q((d + key * s1)* 1/e)#s= k-1(z+rd)(mod n)
        return s1,s2
    
    def verify(self, verify_key, signature, msg):
        V, G = verify_key
        s1, s2 = signature
        Q = GF(G.order())
        
        Z = IntegerRing()
        m = Z(msg.encode("hex"), base=16)
        d = Q(m)
        
        v1 = Q(d * 1/s2)
        v2 = Q(s1 * 1/s2)
        (x,y,z) = (int(v1)*self.G) + (int(v2)*V)
        if Q(x)!=s1:
            raise ValueError("Assinatura Invalida")

In [51]:
ecdsa = ECDSA()
msg = "Esta mensagem vai ser assinada e verificada com sucesso" 
print "Mensagem:", msg

sign_key = ecdsa.sign_key()
verify_key = ecdsa.verify_key()

signature = ecdsa.sign(sign_key,msg)
print "Signature:", signature

msg = ecdsa.verify(verify_key, signature, msg)
print "Verificação Bem Sucedida"

print "-----------------------"

Mensagem: Esta mensagem vai ser assinada e verificada com sucesso
Signature: (12652478553964903363027975263199374617819263728858452934699004753604, 14506590897245168958621185621731102383861829011454942948399060541106)
Verificação Bem Sucedida
-----------------------
